In [1]:
import logging
import importlib
import os, sys
import pandas as pd
import numpy as np

sys.path.insert(0,"/home/eddie_akers_anchorlabs_com/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.environ["HOME"] + "/anchorage/source/python/lib/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility # type: ignore
import utils as aql_utils # type: ignore

#sys.path.append(path + "keys/")
#import keys

#pd.set_option('display.max_columns', None)
#pd.options.display.float_format = '{:,.10f}'.format

In [2]:
google_sheet_secret = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
dashboard_sheet_name = "Copy of A1 - Trading"

talos_adv_api = 'ANC34AV1RAK2'
talos_adv_secret_api =  'mj9a70zpeve9rnam55kn0ql6c7sze3gw'
talos_host_wl = "tal-160.prod.talostrading.com"

#talos_api_sandbox = keys.talos_sandbox_api_key
#talos_secret_api_sandbox = keys.talos_sandbox_api_secret
#host_sandbox = keys.talos_sandbox_host
#talos_sandbox = talos_utils.Talos(talos_api_sandbox, talos_secret_api_sandbox, host_sandbox)



api_key = 'ANCHJEWQR3YH'
api_secret = '3c4nnc7lg7mfg2ae2irz5kkueeei37pz'

talos_wl_api = 'ANC34AV1RAK2'
talos_wl_secret_api = 'mj9a70zpeve9rnam55kn0ql6c7sze3gw'

talos_api = 'ANCHJEWQR3YH'
talos_secret = '3c4nnc7lg7mfg2ae2irz5kkueeei37pz'

host = "tal-42.prod.talostrading.com"
host_wl = "tal-160.prod.talostrading.com"





In [3]:
market_translation = {
    "b2c2/b-2-c-2-a-1" : {"type": "dealer", "full_name" : "B2C2 Overseas, Ltd."},
    "wintermute/wintermute-trading-rfq" : {"type": "dealer", "full_name" : "Wintermute Trading LTD"},
    "okex/okx" : {"type": "exchange", "full_name" : "OKX (Aux Cayes FinTech Co. Ltd.)"},
    "kraken/kraken-a-1" : {"type": "exchange", "full_name" : "Kraken (Payward Trading Ltd.)"},
    "coinbase/coinbase-hrp" : {"type": "exchange", "full_name" : "HRP Coinbase"},
    "bybit/bybit-a-1" : {"type": "exchange", "full_name" : "Bybit (Bybit Fintech Limited)"},
    "galaxy/galaxy-trading-rfq-a-1" : {"type": "dealer", "full_name" : "Galaxy Trading Asia Limited"},
    #sandbox markets
    "coinbase/coinbase" : {"type": "exchange", "full_name" : "Coinbase Hold"},
    "kraken/kraken" : {"type": "exchange", "full_name" : "Kraken Hold"},
    "galaxy/galaxy" : {"type": "dealer", "full_name" : "Galaxy Hold"},
    "jump/jump" : {"type": "dealer", "full_name" : "Jump Hold"},
    "hehmeyer/hehmeyer" : {"type": "dealer", "full_name" : "Hehmeyer Hold"},
    "dvchain/dv-chain" : {"type": "dealer", "full_name" : "DV Chain Hold"},
    "wintermute/wintermute" : {"type": "dealer", "full_name" : "Wintermute Hold"},
}

In [4]:
talos_wl = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, talos_host_wl)
#not booking automatically yet
#gsheet_key = utils.read_secret(google_sheet_secret)

In [5]:
def construct_orders_entries(order_response: list) -> pd.DataFrame:
    order_df = pd.DataFrame(order_response)
    order_id = order_df["OrderID"][0]
    order_denom = order_df["Currency"][0]

    markets_df = pd.DataFrame(order_df["Markets"][0])
    markets_df = markets_df[markets_df["CumQty"].astype(float) > 0]
    is_multileg = markets_df["Market"].str.contains("multileg").any()
    is_perp = markets_df["Symbol"].str.contains("PERP").all()
    multileg_type = ""

    if is_multileg:
        multileg_type = "delta_one" if markets_df["Symbol"].str.contains("PERP").any() else "synthetic"
        markets_df = markets_df[markets_df["Market"] != "multileg"]
        
    markets_df["timestamp"] = pd.to_datetime(order_df["Timestamp"][0]).strftime("%Y-%m-%d %H:%M:%S")
    markets_df["full_name"] = markets_df["MarketAccount"].apply(lambda x: market_translation.get(x, {}).get("full_name", "Market not found"))
    markets_df["type"] = markets_df["MarketAccount"].apply(lambda x: market_translation.get(x, {}).get("type", "Market not found"))
    markets_df["asset_type"] = np.where(markets_df["Symbol"].str.contains("PERP"), "perp", "spot")
    markets_df["Symbol"] = markets_df["Symbol"].str.replace("-", "/")
    markets_df["Side"] = markets_df["Side"].str.upper()
    markets_df["FeeCurrency"] = "" if "FeeCurrency" not in markets_df.columns else markets_df["FeeCurrency"]
    markets_df["fees"] = ""
    markets_df["fees_ccy"] = ""
    markets_df = markets_df.reset_index(drop=True)
    
    if not is_multileg and not is_perp:
        markets_df["dealer_trade_id"] = f"{order_id}_" + markets_df["Market"].astype(str) if len(markets_df) > 1 else order_id
        transacted_df = pd.DataFrame(order_df["TransactedAssets"][0]).set_index("Asset").T
        cols = transacted_df.columns
        transacted_df[cols] = transacted_df[cols].apply(pd.to_numeric, errors='coerce')
        transacted_df = transacted_df.loc[["Amount"]].reset_index(drop=True)

        for col in transacted_df.columns:
            if transacted_df[col].iloc[0] > 0:
                transacted_df["bought_asset"] = [col]
                transacted_df = transacted_df.rename(columns={col: 'bought'})
            elif transacted_df[col].iloc[0] < 0:
                transacted_df["sold_asset"] = [col]
                transacted_df = transacted_df.rename(columns={col: 'sold'})
                transacted_df["sold"] = transacted_df["sold"] * -1


        markets_df = pd.concat([markets_df, transacted_df], axis=1).reset_index(drop=True)
    # if is_perp:
    #     markets_df["dealer_trade_id"] = f"{order_id}_" + markets_df["asset_type"].astype(str) if multileg_type == "delta_one" else f"{order_id}_" + markets_df["Market"].astype(str)
    #     markets_df["bought"] = 0
    #     markets_df["bought_asset"] = 0
    #     markets_df["sold"] = 0
    #     markets_df["sold_asset"] = 0
    else:
        print("here")
        markets_df["dealer_trade_id"] = f"{order_id}_" + markets_df["asset_type"].astype(str) if multileg_type == "delta_one" else f"{order_id}_" + markets_df["Market"].astype(str)
        markets_df["bought"] = 0
        markets_df["bought_asset"] = 0
        markets_df["sold"] = 0
        markets_df["sold_asset"] = 0

    market_len = len(markets_df)


    for i in range(market_len):
        lp_type = markets_df["type"].iloc[i]
        base_qty = float(markets_df["OrderQty"].iloc[i])
        quote_qty = float(markets_df["CumAmt"].iloc[i])
        #base_ccy = markets_df["Currency"].iloc[i]
        quote_ccy = markets_df["AmountCurrency"].iloc[i]
        side = markets_df["Side"].iloc[i]
        bought_amt = float(markets_df["bought"].iloc[i])
        sold_amt = float(markets_df["sold"].iloc[i])
        bought_asset = markets_df["bought_asset"].iloc[i]
        sold_asset = markets_df["sold_asset"].iloc[i]
        avg_px_allin = float(markets_df["AvgPxAllIn"].iloc[i])
        talos_reported_fee = markets_df["CumFee"].iloc[i]
        talos_reported_fee_ccy = markets_df["FeeCurrency"].iloc[i]

        if lp_type == "exchange" and market_len == 1:
            
            if side == "BUY":
                buy_minus_buy = abs(base_qty - bought_amt)
                sell_minus_sell = abs(quote_qty - sold_amt)

                fee = max(buy_minus_buy, sell_minus_sell)
                fee_ccy = bought_asset if buy_minus_buy > sell_minus_sell else sold_asset

            elif side == "SELL":
                buy_minus_buy = abs(quote_qty - bought_amt)
                sell_minus_sell = abs(base_qty - sold_amt)

                fee = max(buy_minus_buy, sell_minus_sell)
                fee_ccy = bought_asset if buy_minus_buy > sell_minus_sell else sold_asset

            markets_df["fees"] = fee
            markets_df["fees_ccy"] = fee_ccy

        elif len(markets_df) > 1:
            
            if side == "BUY":
                markets_df.loc[i, "bought"] = base_qty
                markets_df.loc[i, "bought_asset"] = order_denom
                markets_df.loc[i, "sold"] = quote_qty if not is_multileg else avg_px_allin*base_qty
                markets_df.loc[i, "sold_asset"] = quote_ccy

            elif side == "SELL":
                markets_df.loc[i, "bought"] = quote_qty if not is_multileg else avg_px_allin*base_qty
                markets_df.loc[i, "bought_asset"] = quote_ccy
                markets_df.loc[i, "sold"] = base_qty
                markets_df.loc[i, "sold_asset"] = order_denom


            markets_df.loc[i,"fees"] = talos_reported_fee
            markets_df.loc[i,"fees_ccy"] = talos_reported_fee_ccy
        
    if markets_df["type"].str.contains("dealer").all():
        markets_df["matching_ctpy_id"] = ""
        return markets_df[["dealer_trade_id", "matching_ctpy_id", "timestamp", "full_name", "Side", "Symbol", "bought", "bought_asset", "sold", "sold_asset", "AvgPxAllIn", "fees", "fees_ccy", "CumFee", "FeeCurrency", "OrderQty", "CumQty", "CumAmt"]]

    return markets_df[["dealer_trade_id", "timestamp", "full_name", "Side", "Symbol", "bought", "bought_asset", "sold", "sold_asset", "AvgPxAllIn", "fees", "fees_ccy", "CumFee", "FeeCurrency", "OrderQty", "CumQty", "CumAmt"]]

In [6]:
order_id = "f1d0a5b1-49f9-4426-8a04-2f1570c93e89"

test_order = talos_wl.get_orders(order_id=order_id)
construct_orders_entries(test_order).iloc[:,:12]

ValueError: Columns must be same length as key

In [41]:
def multi_order_fills(all_orders):
    combined_df = pd.DataFrame()  # Initialize an empty dataframe to store the combined data
    for identifier in all_orders:
        # Call run_function for each identifier and get the dataframe
        try:
            df = talos_wl.get_trade_fills(order_id=identifier)
            df['Amount'] = df['Amount'].astype(float)
            df['Fee'] = df['Fee'].astype(float)
            df['Quantity'] = df['Quantity'].astype(float)
            df["amount_less_fees"] = df["Amount"] - df["Fee"]
            # Concatenate the dataframe to the combined dataframe
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            time.sleep(0.100)
        except Exception as e:
            err_str = f"ERROR during get_trade_fills for orderID {identifier}: {str(e)}\n\n"
            print(err_str)
    if "Timestamp" in combined_df.columns:
        combined_df = combined_df.set_index('Timestamp')
        combined_df = combined_df.sort_index()
    return combined_df


In [42]:
all_orders = uuids = [
    "b29ec600-0b35-403f-98dc-a3c961523fb4",
    "32144fc6-09c0-4ecd-b9f5-3a0d195832ae",
    "bed157c1-e7d6-482b-be63-603fb48abaa8"]


In [43]:
multi_order_fills(all_orders).to_csv("multi.csv")

ERROR during get_trade_fills for orderID b29ec600-0b35-403f-98dc-a3c961523fb4: name 'time' is not defined


ERROR during get_trade_fills for orderID 32144fc6-09c0-4ecd-b9f5-3a0d195832ae: name 'time' is not defined


ERROR during get_trade_fills for orderID bed157c1-e7d6-482b-be63-603fb48abaa8: name 'time' is not defined




In [44]:
order_list = all_orders


final_df = pd.DataFrame()

for i in order_list:
    order = talos_wl.get_orders(order_id=i)
    order_df = pd.DataFrame(order)
    datetime = pd.to_datetime(order_df["Timestamp"][0]).strftime("%Y-%m-%d %H:%M:%S")
    order_markets_df = pd.DataFrame(order_df["Markets"][0]).dropna(subset=["LegIndex"])
    order_markets_df["timestamp"] = datetime
    order_markets_df["order_id"] = i

    final_df = pd.concat([final_df, order_markets_df], axis=0)

In [45]:

df = final_df
# Assuming df is your DataFrame
#df['Market'] = df['Market'].replace('galaxy', 'gtc')
df['new_orderID'] = df['order_id'] + "_" + df['Market']
filtered_df = df[df['OrderQty'] != "0"]
# Assuming df is your DataFrame



display(filtered_df)
filtered_df.to_csv('multi_fills.csv')

df = filtered_df
# Step 1: Add a new column with the value "prop_trade"
df['prop_trade'] = "prop_trade"

# Step 2: Reorder the columns
df = df[['new_orderID', 'prop_trade', 'timestamp', 'Market', 'Side', 'CumAmt', 'CumQty', 'AvgPxAllIn','Currency']]

# Step 3: Display the reordered DataFrame
display(df)

,Market,OrdStatus,OrderQty,OrdType,Price,LeavesQty,CumQty,CumAmt,CumFee,AvgPx,...,Side,LegIndex,Currency,AmountCurrency,MarketAccount,AvgPxAllIn,LegSummaryIndex,timestamp,order_id,new_orderID
0,cumberland,Filled,0.07366809,Limit,0,0,0.07366809,4999.9991188414,0,67871.97983335,...,Buy,0,BTC,USD,cumberland/cumberland-trading-rfq,67871.97983335,0,2024-10-16 12:14:04,b29ec600-0b35-403f-98dc-a3c961523fb4,b29ec600-0b35-403f-98dc-a3c961523fb4_cumberland
3,wintermute,Filled,0.07366809,Limit,0,0,0.07366809,5002.21,0,67902.11065763,...,Sell,1,BTC,USD,wintermute/wintermute-trading-rfq,67902.11065763,1,2024-10-16 12:14:04,b29ec600-0b35-403f-98dc-a3c961523fb4,b29ec600-0b35-403f-98dc-a3c961523fb4_wintermute
0,cumberland,Filled,94.10570045,Limit,0,0,94.10570045,14679.7018417249,0,155.99163251,...,Buy,0,SOL,USD,cumberland/cumberland-trading-rfq,155.99163251,0,2024-10-16 12:53:42,32144fc6-09c0-4ecd-b9f5-3a0d195832ae,32144fc6-09c0-4ecd-b9f5-3a0d195832ae_cumberland
2,wintermute,Filled,94.10570045,Limit,0,0,94.10570045,14682.75,0,156.02444183,...,Sell,1,SOL,USD,wintermute/wintermute-trading-rfq,156.02444183,1,2024-10-16 12:53:42,32144fc6-09c0-4ecd-b9f5-3a0d195832ae,32144fc6-09c0-4ecd-b9f5-3a0d195832ae_wintermute
0,cumberland,Filled,16.94038831,Limit,0,0,16.94038831,44713.3921561435,0,2639.45497222,...,Buy,0,ETH,USD,cumberland/cumberland-trading-rfq,2639.45497222,0,2024-10-16 12:54:07,bed157c1-e7d6-482b-be63-603fb48abaa8,bed157c1-e7d6-482b-be63-603fb48abaa8_cumberland
1,galaxy,Filled,7.26642823,Limit,0,0,7.26642823,19195.09863229,0,2641.61401238,...,Sell,1,ETH,USD,galaxy/galaxy-trading-rfq-a-1,2641.61401238,1,2024-10-16 12:54:07,bed157c1-e7d6-482b-be63-603fb48abaa8,bed157c1-e7d6-482b-be63-603fb48abaa8_galaxy
2,wintermute,Filled,9.67396008,Limit,0,0,9.67396008,25531.85,0,2639.24170593,...,Sell,1,ETH,USD,wintermute/wintermute-trading-rfq,2639.24170593,1,2024-10-16 12:54:07,bed157c1-e7d6-482b-be63-603fb48abaa8,bed157c1-e7d6-482b-be63-603fb48abaa8_wintermute


/tmp/ipykernel_119629/1842949004.py:15 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,new_orderID,prop_trade,timestamp,Market,Side,CumAmt,CumQty,AvgPxAllIn,Currency
0,b29ec600-0b35-403f-98dc-a3c961523fb4_cumberland,prop_trade,2024-10-16 12:14:04,cumberland,Buy,4999.9991188414,0.07366809,67871.97983335,BTC
3,b29ec600-0b35-403f-98dc-a3c961523fb4_wintermute,prop_trade,2024-10-16 12:14:04,wintermute,Sell,5002.21,0.07366809,67902.11065763,BTC
0,32144fc6-09c0-4ecd-b9f5-3a0d195832ae_cumberland,prop_trade,2024-10-16 12:53:42,cumberland,Buy,14679.7018417249,94.10570045,155.99163251,SOL
2,32144fc6-09c0-4ecd-b9f5-3a0d195832ae_wintermute,prop_trade,2024-10-16 12:53:42,wintermute,Sell,14682.75,94.10570045,156.02444183,SOL
0,bed157c1-e7d6-482b-be63-603fb48abaa8_cumberland,prop_trade,2024-10-16 12:54:07,cumberland,Buy,44713.3921561435,16.94038831,2639.45497222,ETH
1,bed157c1-e7d6-482b-be63-603fb48abaa8_galaxy,prop_trade,2024-10-16 12:54:07,galaxy,Sell,19195.09863229,7.26642823,2641.61401238,ETH
2,bed157c1-e7d6-482b-be63-603fb48abaa8_wintermute,prop_trade,2024-10-16 12:54:07,wintermute,Sell,25531.85,9.67396008,2639.24170593,ETH


In [46]:
# Step 1: Convert CumAmt and CumQty to numeric
filtered_df['CumAmt'] = pd.to_numeric(filtered_df['CumAmt'], errors='coerce')
filtered_df['CumQty'] = pd.to_numeric(filtered_df['CumQty'], errors='coerce')
filtered_df['AvgPxAllIn'] = pd.to_numeric(filtered_df['AvgPxAllIn'], errors='coerce')

# Calculate the sumproduct for Weighted Avg Price
filtered_df['WeightedPrice'] = filtered_df['AvgPxAllIn'] * filtered_df['CumQty']

# Now aggregate
aggregated_df = filtered_df.groupby('new_orderID').agg(
    CumQty=('CumQty', 'sum'),
    CumAmt=('CumAmt', 'sum'),
    WeightedAvgPx=('WeightedPrice', 'sum')
).reset_index()

# Normalize Weighted Average Price
aggregated_df['WeightedAvgPx'] = aggregated_df['WeightedAvgPx'] / aggregated_df['CumQty']


# Merge back with the original DataFrame to keep other columns
merged_df = pd.merge(aggregated_df, filtered_df[['new_orderID', 'timestamp', 'Market', 'Side','Currency']], on='new_orderID', how='left')

# Keep only the specified columns
final_df = merged_df[['new_orderID', 'timestamp', 'Market', 'Side', 'CumAmt', 'CumQty', 'WeightedAvgPx','Currency']]


# Display the result
print(final_df)



/tmp/ipykernel_119629/3742916774.py:2 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_119629/3742916774.py:3 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_119629/3742916774.py:4 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_119629/3742916774.py:7 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

                                       new_orderID            timestamp  \
0  32144fc6-09c0-4ecd-b9f5-3a0d195832ae_cumberland  2024-10-16 12:53:42   
1  32144fc6-09c0-4ecd-b9f5-3a0d195832ae_wintermute  2024-10-16 12:53:42   
2  b29ec600-0b35-403f-98dc-a3c961523fb4_cumberland  2024-10-16 12:14:04   
3  b29ec600-0b35-403f-98dc-a3c961523fb4_wintermute  2024-10-16 12:14:04   
4  bed157c1-e7d6-482b-be63-603fb48abaa8_cumberland  2024-10-16 12:54:07   
5      bed157c1-e7d6-482b-be63-603fb48abaa8_galaxy  2024-10-16 12:54:07   
6  bed157c1-e7d6-482b-be63-603fb48abaa8_wintermute  2024-10-16 12:54:07   

       Market  Side        CumAmt     CumQty  WeightedAvgPx Currency  
0  cumberland   Buy  14679.701842  94.105700     155.991633      SOL  
1  wintermute  Sell  14682.750000  94.105700     156.024442      SOL  
2  cumberland   Buy   4999.999119   0.073668   67871.979833      BTC  
3  wintermute  Sell   5002.210000   0.073668   67902.110658      BTC  
4  cumberland   Buy  44713.392156  16.940388

In [47]:
df = final_df
# Step 1: Add a new column with the value "prop_trade"
df['prop_trade'] = "prop_trade"

# Step 2: Reorder the columns
df = df[['new_orderID', 'prop_trade', 'timestamp', 'Market', 'Side', 'CumAmt', 'CumQty', 'WeightedAvgPx','Currency']]

# Step 3: Display the reordered DataFrame
display(df)


/tmp/ipykernel_119629/325636533.py:3 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,new_orderID,prop_trade,timestamp,Market,Side,CumAmt,CumQty,WeightedAvgPx,Currency
0,32144fc6-09c0-4ecd-b9f5-3a0d195832ae_cumberland,prop_trade,2024-10-16 12:53:42,cumberland,Buy,14679.701842,94.105700,155.991633,SOL
1,32144fc6-09c0-4ecd-b9f5-3a0d195832ae_wintermute,prop_trade,2024-10-16 12:53:42,wintermute,Sell,14682.750000,94.105700,156.024442,SOL
2,b29ec600-0b35-403f-98dc-a3c961523fb4_cumberland,prop_trade,2024-10-16 12:14:04,cumberland,Buy,4999.999119,0.073668,67871.979833,BTC
3,b29ec600-0b35-403f-98dc-a3c961523fb4_wintermute,prop_trade,2024-10-16 12:14:04,wintermute,Sell,5002.210000,0.073668,67902.110658,BTC
4,bed157c1-e7d6-482b-be63-603fb48abaa8_cumberland,prop_trade,2024-10-16 12:54:07,cumberland,Buy,44713.392156,16.940388,2639.454972,ETH
5,bed157c1-e7d6-482b-be63-603fb48abaa8_galaxy,prop_trade,2024-10-16 12:54:07,galaxy,Sell,19195.098632,7.266428,2641.614012,ETH
6,bed157c1-e7d6-482b-be63-603fb48abaa8_wintermute,prop_trade,2024-10-16 12:54:07,wintermute,Sell,25531.850000,9.673960,2639.241706,ETH
